In [5]:
#from kb import WumpusKB
from aima.logic import expr, dpll_satisfiable
%load_ext autoreload
%autoreload 2
import pdb
import random
import time
import copy
import numpy


In [238]:
        r = random.randint(2, 20)
        if r <= 3:
            i = 1
            j = r+1
        else:
            i = (r + 4) % 5
            j = (r + 1) % 5
        print i,j

3 0


In [6]:
action_list = {
    'forward': 0,
    'left': 1,
    'right': 2,
    'shoot': 3,
    'pick': 4,
    'noop': 5
    }

facing_list = {
    'up': 0,
    'right': 1,
    'down': 2,
    'left': 3
    }

map_list = ['G', 'W', 'S', 'P', 'B']


def neighbors(i, j):
    li = set()
    if (i - 1) in range(5):
        li.add((i-1, j))
    if (i + 1) in range(5):
        li.add((i+1, j))
    if (j - 1) in range(5):
        li.add((i, j-1))
    if (j + 1) in range(5):
        li.add((i, j+1))
    return li

class Agent:

    """AI agent"""
    def _random_ij(self):
        r = random.randint(2, 20)
        if r <= 3:
            i = 0
            j = r+1
        else:
            i = (r - 1) / 4
            j = (r + 1) % 4

        return (i, j)

    def __init__(self):
        self.alive = False
        self.gold_picked = False
        self.all_pos = set()
        for i in range(5):
            for j in range(5):
                self.all_pos.add((i, j))

    def build_world(self):
        self.alive = True
        self.gold_picked = False
        self.pos = (-1, 0)
        self.facing = facing_list['right']
        self.visited = set()
        self.danger = set()
        self.safe = set()
        self.fringe = set()
        self.plan = [action_list['forward']]
        
        self.world = numpy.zeros((5, 5, 5))
        self.known_world = numpy.ones((5, 5, 5))
        self.kb = WumpusKB()

    #Gold
        i, j = self._random_ij()
        self.world[0][i][j] = 2
    #Wumpus
        i, j = self._random_ij()
        self.world[1][i][j] = 2
    #stench
        for pos in neighbors(i, j):
            self.world[2][pos[0]][pos[1]] = 2
    
        for i in range(5):
            for j in range(5):
                if not ((i == 0) and (j == 0)):        
            # generating Pit for this sector with a
            # Probability of 0.2
                    if random.randint(1, 20) <= 1:
                        self.world[3][i][j] = 2
                # generating Breeze
                        for pos in neighbors(i, j):
                            self.world[4][pos[0]][pos[1]] = 2
        #return self.world
    
    def _pl_wumpus_agent(self, percept):
        if len(self.safe) > 0:
            source = self.safe
        elif len(self.fringe) > 0:
            source = self.fringe
        else:
            source = self.danger
        goal = source.pop()
        self.plan = self._path2plan(self._shortest_path(goal))
        action = self.plan.pop()
        print "action: ", action
        return action
    
    def _next(self):

        if self.alive and (not self.gold_picked):
            if len(self.plan) > 0:
                action = self.plan.pop()
            else:
                i, j = (self.pos[0], self.pos[1])
                percept = self.world[i][j]
                action = self._pl_wumpus_agent(percept)
            
            self._do(action)
            
    def _find_safe(self, i, j):
        new = neighbors(i, j) - self.visited
        new -= self.safe
        new -= self.danger

        new_safe = set()
        new_danger = set()

        if (self.world[2][i][j] == 0) and (self.world[4][i][j] == 0):
            self.fringe -= set((i, j))
            new_safe |= new
        else:
            self.fringe |= new

        print "fringe: ", self.fringe

        fringe = self.fringe.copy()
        if len(self.visited) > 1:
            for x in fringe:
                print(x)
                if self.kb.ask(expr('(P%s%s | W%s%s)' % (x*2))):
                    new_danger.add(x)
                    self.fringe.remove(x)
                    
                elif self.kb.ask(expr('(~P%s%s & ~W%s%s)' % (x*2))):
                    new_safe.add(x)
                    self.fringe.remove(x)
        if len(new_safe) > 0:
            self.safe |= (new_safe)
        if len(new_danger) > 0:
            self.danger |= new_danger

        print "safe list: ", self.safe
        
    def _do(self, action):
        facing_dict = {
                facing_list['up']: (0, 1),
                facing_list['right']: (1, 0),
                facing_list['down']: (0, -1),
                facing_list['left']: (-1, 0),
                }
        if action is action_list['forward']:
            dx, dy = facing_dict[self.facing]
            new_pos = (self.pos[0]+dx, self.pos[1]+dy)
            print "new_pos: ", new_pos
            if new_pos in neighbors(*self.pos):

                if new_pos not in self.visited:
                    i, j = new_pos
                    percept = self.world[i][j]
                    print "percept: ",percept

                    self.visited.add((i, j))
                    if (i, j) in self.safe:
                        self.safe.remove((i, j))
                    self.known_world[i][j] = percept

                    if percept[0] == 2:
                        self.gold_picked = True
                        self._do(action_list['pick'])
                        print 'pick'
                    elif (percept[1] == 2) or (percept[3] == 2):
                        self.alive = False
                        print 'die'
                    else:
                        c = 0
                        know = []
                        for x in percept:
                            if x == 2:
                                know.append("%s%s%s" % (map_list[c], i, j))
                            elif x == 0:
                                know.append("~%s%s%s" % (map_list[c], i, j))
                            c += 1
                        if len(know) > 0:
                            print "tell kb: ", ' & '.join(know)
                            self.kb.tell(expr(' & '.join(know)))
                        self._find_safe(i, j)
                self.pos = new_pos
                print "self.pos: ", self.pos
        elif action is action_list['left']:
            self.facing = (self.facing - 1) % 4
            
        elif action is action_list['right']:
            self.facing = (self.facing + 1) % 4
                    
        elif action is action_list['pick']:
            i, j = (self.pos[0], self.pos[1])
            self.world[0][i][j] = 0
            self.gold_picked = True
            
    def _shortest_path(self, goal):
        g = self.visited
        v = {}
        for x in g:
            v[x] = 625
        v[goal] = 0

        def v_neighbors(i, j, s):
            li = []
            for t in neighbors(i, j):
                 if (t in self.visited) and (t not in s):
                    li.append(t)
            return li

        def get_path(v):
            path = []
            a = self.pos
            t = v[a]
            while t > 1:
                for x in v_neighbors(a[0], a[1], s):
                    if v[x] < t:
                        t = t - 1
                        path.append(x)
                        a = x
                        break
            return path

        s = [goal]
        while (len(s) > 0):
            u = s.pop()
            for x in v_neighbors(u[0], u[1], s):
                if v[x] > v[u] + 1:
                    v[x] = v[u] + 1
                    s.append(x)

        pa = get_path(v)
        pa.append(goal)
        print "path: ", pa
        return pa

    def _path2plan(self, path):
        plan = []
        prev = self.pos
        prev_facing = self.facing
        for pos in path:
            dx = pos[0] - prev[0]
            dy = pos[1] - prev[1]
            dy_dict = {
                -1: 'down',
                1: 'up',
                0: 'up' # dummy item
                }
            dx_dict = {
                -1: 'left',
                1: 'right',
                0: dy_dict[dy]
                }
            facing = facing_list[dx_dict[dx]]
            d = facing - prev_facing

            if d != 0:
                c = abs(d)
                if c > 2:
                    d = -d
                    c = c % 2
                if d < 0:
                    direction = action_list['left']
                elif d > 0:
                    direction = action_list['right']

                plan.extend([direction] * c)
                prev_facing = facing
            plan.append(action_list['forward'])
            prev = pos
        plan.reverse()
        print "plan: ", plan
        return plan


In [267]:
for r in range(2, 20)
        #if r <= 3:
            #i = 1
            #j = r+1
        #else:
i = (r + 4) % 5
j = (r + 1) % 5
print(i)
print(j)

(1, 3)
(1, 4)
(1, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 0)


In [255]:
world = numpy.zeros((5, 5))

In [269]:
range(2, 20)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [239]:
b = Agent()

In [247]:
b.build_world()

In [141]:
b._next()

path:  [(1, 2)]
plan:  [0]
action:  0
new_pos:  (1, 2)
percept:  [0. 0. 0. 0. 0.]
tell kb:  ~G12 & ~W12 & ~S12 & ~P12 & ~B12
fringe:  set([(1, 3), (2, 2)])
(1, 3)
ask about:  (P13 | W13)


RuntimeError: maximum recursion depth exceeded while calling a Python object

In [76]:
b.facing = (b.facing - 1) % 4

In [ ]:
b.kb.k& ~s

In [ ]:


a=Agent()
a.build_world()
a.visited = [(0, 0), (1, 0), (2, 0), (0, 1), (0, 2), (1, 2)]
a.pos = (2, 0)
goal = (1, 3)
path = a._shortest_path(goal)
plan = a._path2plan(path)

print "visited :", a.visited
print "cur_pos :", a.pos
print "goal :", goal
print "path :", path

In [ ]:
map_list = ['G', 'W', 'S', 'P', 'B']

In [ ]:
world = b.world

In [248]:
return_world(b.world)

chararray([['e', 'e', 'P', 'e', 'e'],
           ['e', 'G', 'e', 'e', 'e'],
           ['e', 'e', 'e', 'e', 'e'],
           ['e', 'e', 'e', 'e', 'e'],
           ['e', 'e', 'W', 'e', 'e']], dtype='|S1')

In [245]:
def return_world(world):
    world_c = numpy.chararray((5, 5))
    world_c[:] = 'e'
    for i in range(5):
        for j in range(5):
            if world[0][i][j]==2 and world[1][i][j]==0 and world[2][i][j]==0 and world[3][i][j]==0 and world[4][i][j]==0:
                world_c[i][j]='G'
            if world[1][i][j]==2 and world[0][i][j]==0 and world[2][i][j]==0 and world[3][i][j]==0 and world[4][i][j]==0:
                world_c[i][j]='W'
            #if world[2][i][j]==2 and world[0][i][j]==0 and world[1][i][j]==0 and world[3][i][j]==0 and world[4][i][j]==0:
                #world_c[i][j]='S'
            if world[3][i][j]==2 and world[0][i][j]==0 and world[1][i][j]==0 and world[2][i][j]==0 and world[4][i][j]==0:
                world_c[i][j]='P'
            #if world[4][i][j]==2 and world[0][i][j]==0 and world[1][i][j]==0 and world[2][i][j]==0 and world[3][i][j]==0:
                #world_c[i][j]='B'
            if world[0][i][j]==2 and world[1][i][j]==2 and world[2][i][j]==0 and world[3][i][j]==0 and world[4][i][j]==0:
                world_c[i][j]='G+W'
            if world[0][i][j]==2 and world[1][i][j]==0 and world[2][i][j]==0 and world[3][i][j]==2 and world[4][i][j]==0:
                world_c[i][j]='G+P'
            if world[1][i][j]==2 and world[0][i][j]==0 and world[2][i][j]==0 and world[3][i][j]==2 and world[4][i][j]==0:
                world_c[i][j]='W+P'
    return world_c

In [145]:
s = expr('~P00 & ~W00') & ~expr('~P00 & ~W00')
print(s)

((~P00 & ~W00) & ~(~P00 & ~W00))


In [148]:
s.args

[(~P00 & ~W00), ~(~P00 & ~W00)]

In [93]:
#!/usr/bin/env python
class WumpusKB():
    """A KB for wumpus world"""

    def __init__(self, sentence=None):
        self.k = '~P00 & ~W00'

        li = []        
        for i in range(5):
            for j in range(5):
                for l, r in (('B', 'P'), ('S', 'W')):
                    left = "%s%s%s" % (l, i, j)
                    right_list = []
                    for s, t in neighbors(i, j):
                        right_list.append("%s%s%s" % (r, s, t))
                    li.append("(%s <=> (%s))" % \
                              (left, ' | '.join(right_list)))
        e = ' & '.join(li)

        self.tell(e)

        # one and only one wumpus
        li = ['W%s%s' % (i, j) for i in range(5) for j in range(5)]
        e = ' | '.join(li)
 
        self.tell(e)


        li = ['(~W%s%s | ~W%s%s)' % \
              (i, j, x, y)
              for i in range(5) \
              for j in range(5) \
              for x in range(5) \
              for y in range(5) \
              if not ((i == x) and (j == y))]
        e = ' & '.join(li)
        self.tell(e)


    def tell(self, s):
        self.k = '('+self.k+') ' + '&' + '(' + str(s) + ')'

    def ask(self, s):
        print "===================="
        print "ask about: ", s
        start = time.time()
        r = dpll_satisfiable('('+self.k+') ' + '&' + '(' + str(s) + ')')
        #r = 
        print "cost: ", time.time() - start
        if r is False:
            return True
        else:
            return False


In [ ]:
world_c

In [ ]:
map_list = ['G', 'W', 'S', 'P', 'B']